In [4]:
import datetime as dt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
data = pd.read_csv('data/preprocessed_data.csv')

data.head()

,Unnamed: 0,occur_month,occur_day,occur_time,location,victims,day_of_week,crime,neighborhood,hour_0.0,...,month_11,month_12,week_0,week_1,week_2,week_3,week_4,week_5,week_6,week_7
0,0,12,30,23.0,2014,2.0,2,5,78,0,...,0,1,0,0,1,0,0,0,0,0
1,1,12,18,13.0,355,1.0,6,5,115,0,...,0,1,0,0,0,0,0,0,1,0
2,2,12,30,22.0,3473,1.0,2,5,143,0,...,0,1,0,0,1,0,0,0,0,0
3,3,12,30,20.0,2858,1.0,2,5,62,0,...,0,1,0,0,1,0,0,0,0,0
4,4,12,31,0.0,813,2.0,3,5,1,1,...,0,1,0,0,0,1,0,0,0,0


In [7]:
learning_rate = 0.1
num_steps = 500
batch_sizee = 128
display_step = 100

In [11]:
n_hidden_1 = 256
n_hidden_2 = 256
num_input = 83
num_classes = 11

# data.crime.nunique()
X = tf.placeholder(tf.int8, [None, 83])
y = tf.placeholder(tf.int8, [None, 11])

In [7]:
X = data.drop(['crime'], axis=1)
y = data['crime']


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 4)

In [6]:
learning_rate = 0.1
num_steps = 500
batch_size = 128
display_step = 100

n_hidden_1 = 16
n_hidden_2 = 16
num_input = 9
num_classes = 11

tf_X = tf.placeholder("float", [None, num_input])
tf_y = tf.placeholder("float", [None, num_classes])


weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

def neural_net(x):
    layer_1 = tf.add(tf.matmul(x,weights['h1']), biases['b1'])
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer


logit = neural_net(tf_X)
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logit, labels=tf_y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

correct_pred = tf.equal(tf.argmax(logit, 1), tf.argmax(tf_y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

In [7]:
def ceil(a, b):
    return -(-a//b)

In [10]:
with tf.Session() as sess:
    sess.run(init)
    
    for step in range(1, num_steps+1):
        n_sample = len(X_train)
        top_sample = ceil(n_sample, ceil(n_sample, batch_size))
        
        for i in range(ceil(n_sample, top_sample)):
            mini_x = X_train[i * top_sample: (i+1) * top_sample]
            mini_y = y_train[i * top_sample: (i+1) * top_sample]
            sess.run(train_op, feed_dixt={tf_X: mini_x, tf_y: mini_y})
            if step % display_step == 0 or step == 1:
                loss, acc = sess.run([loss_op, accuracy], feed_dict={tf_X: mini_x, tf_Y: mini_y})
                print("Step " + str(step) + ", minibatch loss = " + \
                      "{:.4f}".format(loss) + ", training accuracy = " + \
                      "{:.3f}".format(acc))
                
    print("Optimization finished")
    print("testing accuracy: ", sess.run(accuracy, feed_dict={tf_X: X_test, tf_y: y_test}))

TypeError: run() got an unexpected keyword argument 'feed_dixt'